# EDA

In [1]:
import numpy as np
import pandas as pd

In [2]:
feat = pd.read_csv('features.csv', header=None)
feat.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,195,72,133,246,117,72,72,199,68,36,...,21,0,0,0,0,128,250,226,116,221
1,239,159,239,144,147,0,0,128,147,0,...,0,144,147,0,0,128,147,0,0,64
2,229,32,32,132,226,8,0,81,227,8,...,32,131,5,22,0,0,10,80,0,147
3,3,0,0,0,196,32,96,0,3,0,...,160,4,198,32,96,0,196,0,160,8
4,20,64,0,60,0,0,0,0,131,194,...,24,16,0,0,24,18,0,67,16,42


In [3]:
feat.shape

(238982, 64)

In [4]:
lbl = pd.read_csv('labels.csv', header=None)
lbl.head()

,0
0,x86_64
1,avr
2,arm
3,sparc
4,mips


In [5]:
lbl.shape

(238982, 1)

In [22]:
def get_values(f_path, l_path):
    '''Helper function to get stored values'''
    feat = pd.read_csv(f_path, header=None)
    lbl = pd.read_csv(l_path, header=None)
    return feat.values, lbl.values.reshape(-1)

In [23]:
X, y = get_values('features.csv', 'labels.csv')

---

# Test models

## Split data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Pre-processing

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Scale X

In [10]:
scaler = StandardScaler()

In [11]:
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Encode y

In [12]:
le = LabelEncoder()

In [13]:
y_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

## Model survey

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [15]:
clf1 = LogisticRegression(random_state=42).fit(X_scaled, y_enc)

In [16]:
clf1.score(X_test_scaled, y_test_enc)

0.22967095669815507

In [17]:
clf2 = DecisionTreeClassifier(random_state=42).fit(X_scaled, y_enc)

In [18]:
clf2.score(X_test_scaled, y_test_enc)

0.7475305902491599

In [19]:
clf3 = RandomForestClassifier(random_state=42).fit(X_scaled, y_enc)

In [20]:
clf3.score(X_test_scaled, y_test_enc)

0.9610473594116529

## Random Forest

### Dimensionality reduction

In [24]:
from sklearn.decomposition import PCA

In [27]:
pca = PCA(n_components=10)

In [31]:
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [29]:
clf4 = RandomForestClassifier(random_state=42).fit(X_pca, y_enc)

In [32]:
clf4.score(X_test_pca, y_test_enc)

0.6168262220249794

PCA did not decrease the training time by much, and the results are much worse.

### Hyperparameter tuning

In [34]:
from sklearn.model_selection import RandomizedSearchCV

In [35]:
param_grid = {
    'n_estimators':      [int(x) for x in np.linspace(100, 1000, 10)],
    'max_depth':         [int(x) for x in np.linspace(10, 100, 10)] + [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf':  [1, 2, 4],
    'max_features':      ['auto', 'sqrt'],
    'bootstrap':         [True, False],
}

In [36]:
clf = RandomForestClassifier()

In [80]:
clf_rand = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_grid,
    n_iter=100,
    cv=3,
    n_jobs=-1,
    verbose=5,
    random_state=42)

In [81]:
# This is actually overkill. The default hyperparameters are pretty good.

clf_rand.fit(X_scaled, y_enc)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

### Train full dataset

In [48]:
X, y = get_values('features.csv', 'labels.csv')

# we could use pipelines if this was more complex...
scaler = StandardScaler().fit(X)
enc = LabelEncoder().fit(y)

X_scaled = scaler.transform(X)
y_enc = enc.transform(y)

clf = RandomForestClassifier().fit(X_scaled, y_enc)
clf.score(X_scaled, y_enc)

0.9999832623377493

---

## Let's do it!

In [71]:
from machine_server import Server

In [77]:
s = Server()

tries = 0
while not s.hash:
    s.get()

    x = scaler.transform([[b for b in s.binary]])
    y = enc.inverse_transform(clf.predict(x))[0]

    s.post(y)

    s.log.info("Guess:[{: >9}]   Answer:[{: >9}]   Wins:[{: >3}]".format(y, s.ans, s.wins))
    
    tries += 1
    
print(tries)
print(s.hash)

2021-04-15 15:49:40,195 - machine_server - INFO - Guess:[      avr]   Answer:[      avr]   Wins:[  1]
2021-04-15 15:49:40,405 - machine_server - INFO - Guess:[     m68k]   Answer:[     m68k]   Wins:[  2]
2021-04-15 15:49:40,588 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[  3]
2021-04-15 15:49:40,743 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[  4]
2021-04-15 15:49:40,924 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[  5]
2021-04-15 15:49:41,145 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[  6]
2021-04-15 15:49:41,328 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[  7]
2021-04-15 15:49:41,491 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[  8]
2021-04-15 15:49:41,683 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[  9]
2021-04-15 15:49:41,982 - machine_server - INFO - Guess:[      arm]   Answer:[    

2021-04-15 15:49:56,468 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[ 81]
2021-04-15 15:49:56,643 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[ 82]
2021-04-15 15:49:56,822 - machine_server - INFO - Guess:[      sh4]   Answer:[      sh4]   Wins:[ 83]
2021-04-15 15:49:57,349 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[ 84]
2021-04-15 15:49:57,584 - machine_server - INFO - Guess:[      sh4]   Answer:[      arm]   Wins:[ 84]
2021-04-15 15:49:57,816 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[ 85]
2021-04-15 15:49:58,021 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[ 86]
2021-04-15 15:49:58,259 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[ 87]
2021-04-15 15:49:58,450 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[ 88]
2021-04-15 15:49:58,635 - machine_server - INFO - Guess:[   xtensa]   Answer:[   x

2021-04-15 15:50:12,723 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[161]
2021-04-15 15:50:12,928 - machine_server - INFO - Guess:[     m68k]   Answer:[     m68k]   Wins:[162]
2021-04-15 15:50:13,188 - machine_server - INFO - Guess:[      avr]   Answer:[      avr]   Wins:[163]
2021-04-15 15:50:13,406 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[164]
2021-04-15 15:50:13,943 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[165]
2021-04-15 15:50:14,178 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[166]
2021-04-15 15:50:14,370 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[167]
2021-04-15 15:50:14,616 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[168]
2021-04-15 15:50:14,789 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[169]
2021-04-15 15:50:14,971 - machine_server - INFO - Guess:[      sh4]   Answer:[    

2021-04-15 15:50:29,488 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[240]
2021-04-15 15:50:29,646 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[241]
2021-04-15 15:50:29,839 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[242]
2021-04-15 15:50:29,991 - machine_server - INFO - Guess:[      avr]   Answer:[      avr]   Wins:[243]
2021-04-15 15:50:30,164 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[244]
2021-04-15 15:50:30,347 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[245]
2021-04-15 15:50:30,557 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[246]
2021-04-15 15:50:30,885 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[247]
2021-04-15 15:50:31,128 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[248]
2021-04-15 15:50:31,277 - machine_server - INFO - Guess:[      avr]   Answer:[    

2021-04-15 15:50:46,682 - machine_server - INFO - Guess:[     m68k]   Answer:[     m68k]   Wins:[317]
2021-04-15 15:50:47,022 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[318]
2021-04-15 15:50:47,208 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[319]
2021-04-15 15:50:47,431 - machine_server - INFO - Guess:[     m68k]   Answer:[     m68k]   Wins:[320]
2021-04-15 15:50:47,642 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[321]
2021-04-15 15:50:47,821 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[322]
2021-04-15 15:50:48,019 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[323]
2021-04-15 15:50:48,314 - machine_server - INFO - Guess:[   x86_64]   Answer:[   x86_64]   Wins:[324]
2021-04-15 15:50:48,490 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[325]
2021-04-15 15:50:48,638 - machine_server - INFO - Guess:[  powerpc]   Answer:[  po

...

2021-04-15 16:04:38,591 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4128]
2021-04-15 16:04:38,768 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4129]
2021-04-15 16:04:38,944 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[4130]
2021-04-15 16:04:39,114 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[4131]
2021-04-15 16:04:39,275 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[4132]
2021-04-15 16:04:39,541 - machine_server - INFO - Guess:[   xtensa]   Answer:[   xtensa]   Wins:[4133]
2021-04-15 16:04:39,697 - machine_server - INFO - Guess:[    sparc]   Answer:[    sparc]   Wins:[4134]
2021-04-15 16:04:39,855 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4135]
2021-04-15 16:04:40,031 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[4136]
2021-04-15 16:04:40,273 - machine_server - INFO - Guess:[  powerpc]   Ans

2021-04-15 16:04:58,096 - machine_server - INFO - Guess:[     m68k]   Answer:[     m68k]   Wins:[4204]
2021-04-15 16:04:58,333 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4205]
2021-04-15 16:04:58,491 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4206]
2021-04-15 16:04:58,670 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4207]
2021-04-15 16:04:58,913 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[4208]
2021-04-15 16:04:59,114 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4209]
2021-04-15 16:04:59,284 - machine_server - INFO - Guess:[    sparc]   Answer:[    sparc]   Wins:[4210]
2021-04-15 16:04:59,510 - machine_server - INFO - Guess:[    sparc]   Answer:[    sparc]   Wins:[4211]
2021-04-15 16:04:59,727 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[4212]
2021-04-15 16:04:59,905 - machine_server - INFO - Guess:[   mipsel]   Ans

2021-04-15 16:05:15,529 - machine_server - INFO - Guess:[      avr]   Answer:[      avr]   Wins:[4279]
2021-04-15 16:05:15,881 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[4280]
2021-04-15 16:05:16,037 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[4281]
2021-04-15 16:05:16,245 - machine_server - INFO - Guess:[  powerpc]   Answer:[  powerpc]   Wins:[4282]
2021-04-15 16:05:16,398 - machine_server - INFO - Guess:[    sparc]   Answer:[    sparc]   Wins:[4283]
2021-04-15 16:05:16,566 - machine_server - INFO - Guess:[      arm]   Answer:[      arm]   Wins:[4284]
2021-04-15 16:05:16,752 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[4285]
2021-04-15 16:05:16,997 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4286]
2021-04-15 16:05:17,250 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[4287]
2021-04-15 16:05:17,467 - machine_server - INFO - Guess:[      arm]   Ans

2021-04-15 16:05:31,583 - machine_server - INFO - Guess:[   mipsel]   Answer:[   mipsel]   Wins:[4358]
2021-04-15 16:05:31,789 - machine_server - INFO - Guess:[    sparc]   Answer:[    sparc]   Wins:[4359]
2021-04-15 16:05:31,966 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[4360]
2021-04-15 16:05:32,243 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4361]
2021-04-15 16:05:32,437 - machine_server - INFO - Guess:[     mips]   Answer:[     mips]   Wins:[4362]
2021-04-15 16:05:32,625 - machine_server - INFO - Guess:[alphaev56]   Answer:[alphaev56]   Wins:[4363]
2021-04-15 16:05:32,792 - machine_server - INFO - Guess:[      avr]   Answer:[      avr]   Wins:[4364]
2021-04-15 16:05:33,096 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4365]
2021-04-15 16:05:33,334 - machine_server - INFO - Guess:[     s390]   Answer:[     s390]   Wins:[4366]


KeyboardInterrupt: 